In [ ]:
## Get the raw text file
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

## Section 2.2: Tokenizing text

In [35]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
print('Total number of characters:', len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [36]:
import re

text = "Hello world! This is a test."

#split
result = re.split(r'\s',text)
print(result)

#split using punctuation
result = re.split(r'([,.]|\s)',text)
print(result)

#remove empties
result = [item for item in result if item.strip()]
print(result)

#more general punctuation issues:
text = 'Hello, world. Is this-- a test?'

result = re.split(r'([,.:;?_!"()\']|--|\s)',text)
result = [item for item in result if item.strip()]
print(result)



['Hello', 'world!', 'This', 'is', 'a', 'test.']
['Hello', ' ', 'world!', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
['Hello', 'world!', 'This', 'is', 'a', 'test', '.']
['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [37]:
#now preprocess our raw text file
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


In [47]:
#build vocabulary
all_words = sorted(set(preprocessed))    
vocab_size = len(all_words)
print('Vocab size:',vocab_size)

#assign numeric ids to words
vocab = {token:i for i,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

Vocab size: 1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


## Section 2.3 Converting tokens into token IDs

In [ ]:
#now build a tokenizer class
class SimpleTokenizerV1:
    def __init__(self, vocab):
        #store vocab as member
        self.str_to_int = vocab
        #create inverse lookup table as member too
        self.int_to_str = {i:s for s,i in vocab.items()}

    #input: text
    #output: vector of ids
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        return [self.str_to_int[token] for token in preprocessed]
        
        
    #input: vector of ids
    #output: text sequence (str)
    def decode(self, ids):
        text = " ".join([self.int_to_str[id] for id in ids])
        #for tokens that are punctuation, remove preceeding space
        #substitute space + punctuation for punctuation (\1)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1',text)
        return text

In [49]:
#test the tokenzier class
tokenizer = SimpleTokenizerV1(vocab)
#sample text from book:
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)

print('Encoding')
print(ids)

print('Decoding')
print(tokenizer.decode(ids))

print('Self-consistency check')
print(tokenizer.decode(tokenizer.encode(text)))

Encoding
[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
Decoding
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.
Self-consistency check
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## Section 2.4: Special context tokens

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>','<|unk|>'])